<a href="https://colab.research.google.com/github/anmolsriv/COMP/blob/main/DBImpSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A5: Imperative SQL

The goal of this assignment is to develop skills writing functions and triggers. These will add functionality to our database. 

## What to turn in
You must turn in a Jupyter Notebook on Canvas. Basically, we want to be able to hit execute and run your code to create your imperative SQL and run it. **Make sure your cells run without error, even if you have no answer.** 
This  means that any comments or text answers in your file should be in SQL comments or in markdown cells. Answers in the comments section of assignment submissions on CANVAS are not acceptable and will not be read.

For your assignment to be graded, you must include:
* Your COMMENTED code (minimal or uncommented code will lose points)
* The results generated by your code
* DROP statements **at the end of your code** to clean up ALL VIEWs, FUNCTIONs, TRIGGERS, and TABLEs your code creates, excluding the tables we provide.



## Academic Honesty
The following level of collaboration is allowed on this assignment: 

You may discuss the assignment with your classmates at a high level. Any issues getting PostrgreSQL running is totally fine. What is not allowed is direct examination of anyone else's SQL code (on a computer, email, whiteboard, etc.) or allowing anyone else to see your SQL code.

You may use the search engine of your choice to look up the syntax for SQL commands or PostgreSQL imperative SQL syntax, but may not use it to find answers. Be sure to include the urls of any sites you reviewed as comments in your code.


## Grading
The number of points for each query is indicated in the question.   
If you don't get the right answer or your code is not correct, you won't get all of the points; partial credit may be given at the discretion of the grader.

This is intended to be an SQL assignment. You may use VIEWs as needed and you may use standard built-in PostgreSQL functions (e.g. `ROUND`, `IF`, or `CASE` statements). If you're not sure if something is allowed, ask!


In order for our autograder to work, we need to connect your solution to a local database. 

**DO NOT RUN CELLS THAT START WITH THE COMMENT `# LOCALDB` and do not remove those cells.**

For cells that contain your answer, replace the `SELECT 1` SQL code with your solution. 

**Do not change the first line of the cell or remove the following cell.  Do not add any text (comments, etc.) before the first line we provide.** If you do any of these things, the autograder will not work, we will have to manually modify your submission, and you will lose points.


Cells that start with:
`%%sql py_var_x <<`
Redirect the output of that cell to the variable named `py_var_x`. Note that if the next thing in this cell is a comment, you will get an error. Put some SQL code first, then add your comment. We then convert this variable to a dataframe, and record that as your answer for the question. You may add other cells to the notebook, but be sure that your answer is in the cell that copies the result to the appropriate Python variable.


We are using an autograder for the first pass grading. For the autograder to work, you need to complete your work in the appropriate cells and name and order the attributes in your results as directed. 

**It's critical that you use the attribute names and sort order as directed. If attribute name(s) are not specfied, use the name in the source table.**

* Use the function and trigger names specified
* Load data as provided
* We will be running the queries we gave you as well as additional queries that test how well your code works


Do not delete or modify the next 3 cells!

In [ ]:
# LOCALDB
import ipynbname
import psycopg2
from configparser import ConfigParser

def config(filename='.pg_service.conf', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

params = config()
# build the connection string
def make_conn_str(params):

    return f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['dbname']}"
        
# connect to the database
conn_str = make_conn_str(params)
%load_ext sql
%sql $conn_str 
%config SqlMagic.displaylimit=100


# load the data
%system
myCmd = 'psql ' + params['user'] + ' -d ' +params['dbname'] + ' -p ' + params['port'] + ' -f DBImpSQLdata.sql'
!{myCmd}

In [ ]:
# LOCALDB
nb_fname = ipynbname.name()
myInsert = "INSERT INTO studentNotebook(name) VALUES ( '{}' );".format(nb_fname)
print(myInsert)
myCmd = 'psql ' + params['user'] + ' -d ' +params['dbname'] + ' -p ' + params['port']  + ' -c "' + myInsert + '"'
!{myCmd}


## Initialization

The next cell needs to be run each time you start up Google Colab to start up PostgreSQL.

In [142]:
# install
!pip install --upgrade pip
!pip install SQLAlchemy==1.3.23
!pip install psycopg2-binary
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.autolimit=100
# Limit queries to 100 results. Increase this value if needed, but recognize that your notebook will increase in size as well. %config SqlMagic.displaylimit=100
%sql postgresql+psycopg2://@/postgres
!sudo -u postgres createdb ricedb

 * Starting PostgreSQL 10 database server
   ...done.
ERROR:  role "root" already exists
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
createdb: database creation failed: ERROR:  database "ricedb" already exists


The following tables are used in this assignment:

* Assembly(<ins>assemblyId</ins>, productCode, ingName, qty, unit, hasChoice, detail)
    - Assigns ingredients to products
* Ingredient(<ins>ingName</ins>)
    - Contains ingredients that combine into products
* IngChoice(<ins>ingName</ins>, <ins>choiceName</ins>)
    - Contains ingredients that have user selected choices
* Menu(<ins>menuName</ins>)
    - Contains name of the menus
* MenuProduct(<ins>menuName</ins>, <ins>productCode</ins>, price)
    - Associates products to menus with a price for each product on that menu. A single product may be priced differently on different menus.
* Msg(<ins>id</ins>, occurred, msg) 
    - Table in which values can be written for debugging purposes* Product(<ins>productCode</ins>, productName)
    - Contains the names and codes for products
* ProductSold(<ins>productSoldId</ins>, productCode, ticketId, price)
    - Information about the product that was actually sold
* SaleRecord(<ins>saleRecordId</ins>, productSoldId, ingName, qty, unit, detail)
    - Information about each ingredient used in ProductSold
* Ticket(<ins>ticketId</ins>, eventId, ticketTime, numProducts, totalPrice)
    - Contains products sold together by one purchaser
* TruckEvent(<ins>eventId</ins>, eventName, eventStart, eventPlannedEnd, eventActualEnd, eventStatus, menuName totalDollarSales, totalNumProducts, totalNumTickets) 
    - Contains events that the ice cream truck goes to


## Load the data

In [143]:
# create the tables and load the starting data
!psql postgres -f DBImpSQLdata.sql

psql:DBImpSQLdata.sql:1: NOTICE:  drop cascades to 3 other objects
DETAIL:  drop cascades to constraint menuproduct_productcode_fkey on table menuproduct
drop cascades to constraint assembly_productcode_fkey on table assembly
drop cascades to constraint productsold_productcode_fkey on table productsold
DROP TABLE
psql:DBImpSQLdata.sql:2: NOTICE:  drop cascades to 2 other objects
DETAIL:  drop cascades to constraint menuproduct_menuname_fkey on table menuproduct
drop cascades to constraint truckevent_menuname_fkey on table truckevent
DROP TABLE
DROP TABLE
psql:DBImpSQLdata.sql:4: NOTICE:  drop cascades to constraint ticket_eventid_fkey on table ticket
DROP TABLE
psql:DBImpSQLdata.sql:5: NOTICE:  table "employee" does not exist, skipping
DROP TABLE
psql:DBImpSQLdata.sql:6: NOTICE:  table "eventassignment" does not exist, skipping
DROP TABLE
psql:DBImpSQLdata.sql:7: NOTICE:  drop cascades to constraint productsold_ticketid_fkey on table productsold
DROP TABLE
psql:DBImpSQLdata.sql:8: NOTI

### Declarative SQL

1.  (3 points) Write a query that returns the ingredients used in the product named "brain freeze". For each ingredient, return the ingredient name, detail, quantity, unit, whether or not the ingredient requires a choice and all possible choices. Sort by ingredient name then choice name. 

Each ingredient with a choice will be repeated with each choice option. For example, if a product includes ice cream, and the choices are Chocolate, Vanilla, and Strawberry, your query should return 3 records for ice cream in that product.

Show your query results. 

In [144]:
%%sql py_var_IngUsed <<
SELECT a.ingName as name, a.detail, a.qty as quantity, a.unit, a.hasChoice, COALESCE(ingChoice.choiceName, '') AS choice
FROM Product prod 
  JOIN Assembly a
    ON a.productCode = prod.productCode 
  LEFT OUTER JOIN IngChoice ingChoice 
    ON ingChoice.ingName = a.ingName
where prod.productName = 'brain freeze' 
ORDER BY a.ingName, ingChoice.choiceName;

 * postgresql+psycopg2://@/postgres
20 rows affected.
Returning data to local variable py_var_IngUsed


In [145]:
# do not modify or delete this cell
df_q01 = py_var_IngUsed.DataFrame() # Save the result of this task for grading purposes
df_q01

,name,detail,quantity,unit,haschoice,choice
0,cup,20 oz,1.00,item,0,
1,flavor,,1.00,ounce,1,blue raspberry
2,flavor,,1.00,ounce,1,bubble gum
3,flavor,,1.00,ounce,1,cherry
4,flavor,,1.00,ounce,1,grape
5,flavor,,1.00,ounce,1,lemon
6,flavor,,1.00,ounce,1,lime
7,flavor,,1.00,ounce,1,pineapple
8,flavor,,1.00,ounce,1,rootbeer
9,flavor,,1.00,ounce,1,strawberry


2. (3 points) We want to know how much of our inventory we used during an event.

Create a VIEW named `eventIngredient` that returns all of the ingredients used by event, grouped by event, ingName, unit, and detail that shows the number of each ingredient consumed and the total quantity.  This should be a list with a line for every ingredient option that was used during each event. 

Attributes, should include, in this order:

* eventId
* ingName
* unit
* detail
* total number of that ingredient (named totalNumber)
* total quantity of that ingredient (named totalQty)

In [146]:
%%sql
CREATE OR REPLACE VIEW eventIngredient AS
SELECT ticket.eventID, saleRec.ingName, saleRec.unit, saleRec.detail, count(*) AS totalNumber, sum(qty) AS totalQty 
FROM Ticket ticket 
  JOIN ProductSold prodSold 
    ON prodSold.ticketID = ticket.ticketID 
  JOIN SaleRecord saleRec 
    ON saleRec.productSoldID = prodSold.productSoldID 
GROUP BY ticket.eventID, saleRec.ingName, saleRec.unit, saleRec.detail;

 * postgresql+psycopg2://@/postgres
Done.


[]

In [147]:
%%sql
select * from eventIngredient order by eventid;

 * postgresql+psycopg2://@/postgres
0 rows affected.


eventid,ingname,unit,detail,totalnumber,totalqty


## Functions

3. (6 points) Create a function, ``createTicket`` that:
		* Takes as parameters a TruckEvent eventId
		* Creates an empty ticket with the current timestamp
		* Returns the id of the new ticket
		* If there is no event with the specified eventId, returns -1

Notes: Recall that you can use  the keyword `RETURNING` as part of an `INSERT` statement to get back the newly assigned ticketId. See https://www.postgresql.org/docs/10/sql-insert.html for more details.


In [148]:
%%sql 
CREATE OR REPLACE FUNCTION createTicket ( thisEventId INTEGER) 
  RETURNS INTEGER AS  
$$
DECLARE
  newTicketID INTEGER;
BEGIN
  -- your code here, be sure to update the return value

  INSERT INTO Ticket(eventID, ticketTime) 
  VALUES (thisEventId, CURRENT_TIMESTAMP) 
  RETURNING ticketID 
  INTO newTicketID;

  IF
    newTicketID IS NOT NULL
  THEN
  ELSE
    newTicketID = -1;
  END IF;

RETURN newTicketID;
END;
$$
LANGUAGE plpgsql;

  

 * postgresql+psycopg2://@/postgres
Done.


[]

4. (9 points) Write a function, `addProductToTicket`, that takes a ticketId and a productCode and adds the specified product to the specified ticket and returns the id of the new ProductSold record. Be sure to populate all of the appropriate fields for ProductSold (e.g. price).   This function should add all the ingredients **without** choices to the saleRecord table. 

If the product specified is not on the menu for this event, print the message ''Product <XX> is not available'', where XX is the product specified, and return -1. Also return -1 if the ticketId is not valid.

In [149]:
%%sql 
CREATE OR REPLACE FUNCTION addProductToTicket ( thisTicketId INTEGER, thisProductCode CHAR(3)) 
  RETURNS INTEGER AS  
$$
DECLARE 
  productPrice decimal(5, 2);
  ticket_row RECORD;
  newProductSoldID INTEGER;
BEGIN
  newProductSoldID = -1;
  -- your code here, be sure to update the return value

  -- fetching the ticket row
  SELECT * 
  FROM Ticket 
  WHERE ticketID = thisTicketId
  INTO ticket_row;

  IF
    ticket_row IS null
  THEN
    RETURN -1;
  ELSE
  END IF;

  -- get the product price from the menu
  SELECT mp.price 
  FROM MenuProduct mp 
    JOIN TruckEvent te 
      ON mp.menuName = te.menuName
    JOIN Ticket t
      ON t.eventID = te.eventID
  WHERE mp.productCode = thisProductCode AND t.ticketID = thisTicketId
  INTO productPrice;

  IF
    productPrice IS null
  THEN
    RAISE NOTICE 'Product % is not available', thisProductCode ;
    RETURN -1;
  ELSE
  END IF;  

  -- insert into ProductSold table
  INSERT INTO ProductSold(productCode, ticketId, price) 
  VALUES (thisProductCode, thisTicketId, productPrice) 
  RETURNING productSoldID
  INTO newProductSoldID;

  -- insert into SaleRecord table
  INSERT INTO SaleRecord(productSoldId, ingName, qty, unit, detail) 
  (
    SELECT newProductSoldID, a.ingName, a.qty, a.unit, a.detail
    FROM Assembly a
    where a.productCode = thisProductCode AND a.hasChoice=0
  );


RETURN newProductSoldID;
END;
$$
LANGUAGE plpgsql;

  

 * postgresql+psycopg2://@/postgres
Done.


[]

5. (10 points) Write a function, `addIngChoiceToProductSold`, that takes a productSoldId, an ingredient name, a quantity, a unit, and a choice name and adds the chosen ingredient to the specified productSold. Be sure to populate all of the fields for SaleRecord. If an invalid productSoldId is passed in or the specified ingredient is not part of the product, return -1. Otherwise, return the id of the new SaleRecord.

In [150]:
%%sql 
CREATE OR REPLACE FUNCTION addIngChoiceToProductSold ( thisProductSoldId INTEGER, thisIngName VARCHAR(50), 
                                                      thisQty NUMERIC(10,2), thisUnit VARCHAR(20), thisChoice VARCHAR(50)) 
  RETURNS INTEGER AS
$$
DECLARE
  prductSoldRecord RECORD;
  ingredientAssemblyRecord RECORD;
  newSaleRecordID INTEGER;
BEGIN
-- your code here, be sure to update the return value
  newSaleRecordID = -1;
  -- validate the productSold
  SELECT * 
  FROM ProductSold
  WHERE productSoldID = thisProductSoldId 
  INTO prductSoldRecord;

  IF
    prductSoldRecord IS null
  THEN
    RETURN -1;
  ELSE
  END IF;

  -- validate ingredient info
  SELECT a.*
  FROM Assembly a
  where a.productCode = prductSoldRecord.productCode and a.ingName = thisIngName
  INTO ingredientAssemblyRecord;

  IF
    ingredientAssemblyRecord IS null
  THEN
    RETURN -1;
  ELSE
  END IF;

  -- insert into SaleRecord table
  INSERT INTO SaleRecord(productSoldId, ingName, qty, unit, detail) 
  VALUES (thisProductSoldId, thisIngName, thisQty, thisUnit, thisChoice)
  RETURNING newSaleRecordID 
  INTO newSaleRecordID;


RETURN newSaleRecordID;
END;
$$
LANGUAGE plpgsql;

  

 * postgresql+psycopg2://@/postgres
Done.


[]

6. (5 points)  Write a function named `updateEventSales` that takes a truckEvent eventId and updates totalDollarSales, totalNumProducts, and totalNumTickets based on the products sold during the event.   Returns the eventId if successful. Returns -1 if the eventId does not exist.

In [172]:
%%sql 
CREATE OR REPLACE FUNCTION updateEventSales ( thisEventId INTEGER) 
  RETURNS INTEGER AS  
$$
DECLARE
  truckEventRecord RECORD;
  eventAggregateRecord RECORD;
BEGIN
-- your code here, be sure to update the return value
  -- validate the productSold
  SELECT * 
  FROM TruckEvent
  WHERE eventID = thisEventId 
  INTO truckEventRecord;

  IF
    truckEventRecord IS null
  THEN
    RETURN -1;
  ELSE
  END IF;

  -- aggregate event data
  SELECT count(ticketID) as totalNumTickets, COALESCE(sum(numProducts), 0) as totalNumProducts, coalesce(sum(totalPrice), 0) as totalDollarSales
  FROM Ticket
  WHERE eventID = thisEventId 
  INTO eventAggregateRecord;

  RAISE NOTICE 'aggregated record: %', eventAggregateRecord;

  -- update the event record
  UPDATE TruckEvent
  SET totalDollarSales = eventAggregateRecord.totalDollarSales, totalNumProducts = eventAggregateRecord.totalNumProducts, totalNumTickets = eventAggregateRecord.totalNumTickets
  WHERE eventID = thisEventId;

RETURN thisEventId;
END;
$$
LANGUAGE plpgsql;

  

 * postgresql+psycopg2://@/postgres
Done.


[]

7. (25 points) Write a function ``findLongestChain`` that takes an eventId and a product code. ``findLongestChain`` should look at tickets from the specified event in order by their  ticketTime and return the maximum number of tickets in a row that has at least one of the specified products.  You should not make any assumptions about the assignment of productSoldId values. Return -1 if the eventId is invalid. 

For example, say tickets 101, 102, and 103 occured during event 1.

Ticket 101 contains: c1, c2, and c3.

Ticket 102 contains: c2, ms, and dk.

Ticket 103 contains: dk and c1.

If we are looking for the longest chain for product `dk`, the answer would be 2.
The longest chain for c1 is 1.

In [152]:
%%sql 
CREATE OR REPLACE FUNCTION findLongestChain (  thisEventId INTEGER, thisProductCode CHAR(3)) 
  RETURNS INTEGER AS
$$
DECLARE
  truckEventRecord RECORD;
  rowData RECORD;
  longestChain INTEGER;
  currLongestChain INTEGER;
  prevTicketID INTEGER;
  productFoundForTicket INTEGER;
BEGIN
-- your code here, be sure to update the return value
  longestChain = 0;
  currLongestChain = 0;
  -- validate the productSold
  SELECT * 
  FROM TruckEvent
  WHERE eventID = thisEventId 
  INTO truckEventRecord;

  IF
    truckEventRecord IS null
  THEN
    RETURN -1;
  ELSE
  END IF;

  prevTicketID = -1;
  productFoundForTicket = 0;
  FOR rowData IN 
    SELECT t.ticketID, ps.productCode
    FROM Ticket t 
      JOIN ProductSold ps 
        ON ps.ticketID = t.ticketID 
    WHERE t.eventID = thisEventId 
    ORDER BY t.ticketTime, t.ticketID
  LOOP
    -- check if its a new ticketID
    IF 
      rowData.ticketId = prevTicketID 
    THEN
      -- same ticketID
    ELSE
      -- its a new ticket ID
      -- reset current chain length if the chain is broken
      IF 
        productFoundForTicket = 0
      THEN
        -- chain broken
        currLongestChain = 0;
      ELSE
      END IF;
      productFoundForTicket = 0;
      prevTicketID = rowData.ticketId;
    END IF;

    IF 
      productFoundForTicket = 1
    THEN
      -- do nothing
    ELSE
      IF 
        rowData.productCode = thisProductCode
      THEN
        -- product found
        productFoundForTicket = 1;
        currLongestChain = (currLongestChain + 1);
        -- update the longest chain length if applicable
        IF
          currLongestChain > longestChain
        THEN
          longestChain = currLongestChain;
        ELSE
        END IF;
      ELSE
        -- do nothing
      END IF;
    END IF;
  END LOOP;

RETURN longestChain;
END;
$$
LANGUAGE plpgsql;

  

 * postgresql+psycopg2://@/postgres
Done.


[]

## Triggers

8. (10 points)  Write a trigger named `updateTicket` on the productSold table that  updates the numProducts and totalPrice fields in Ticket when a product is added to the productSold table

In [153]:
%%sql 

CREATE OR REPLACE FUNCTION  
   updateTicketInfo() RETURNS TRIGGER AS
$$
BEGIN
  UPDATE Ticket 
  SET numProducts = numProducts + 1, totalPrice = totalPrice + NEW.price
  WHERE ticketID = NEW.ticketId;
   RETURN NEW;
END;
$$
LANGUAGE plpgsql;

CREATE TRIGGER updateTicket 
AFTER INSERT ON ProductSold 
FOR EACH ROW 
EXECUTE PROCEDURE updateTicketInfo();

 * postgresql+psycopg2://@/postgres
Done.
Done.


[]

## Putting it all together with Transactions

9. (1 point) Create an event 'DB study group' with start date time December 1, 2021 at 8 PM and planned end time December 2, 2021 at 12 AM. The status should be 'Scheduled' and the menu should be 'Full menu'.

Assign this event eventId 5. Note that you can assign values to attributes of type `SERIAL` as long as that value does not already exist in the table.


In [154]:
%%sql 
INSERT INTO TruckEvent (eventId, eventName, eventStart, eventPlannedEnd, eventStatus, menuName)
VALUES (5, 'DB study group', '2021-12-01 20:00:00', '2021-12-02 12:00:00', 'Scheduled', 'Full menu');

 * postgresql+psycopg2://@/postgres
1 rows affected.


[]

10. Run your function ```updateEventSales``` for this event to initialize the derived attributes.

In [173]:
%%sql
SELECT updateEventSales(5)

 * postgresql+psycopg2://@/postgres
1 rows affected.


updateeventsales
5


11. (14 points) Create a transaction that includes the following actions:

1 Create a new ticket for this event. Keep track of the unique id for this ticket. There are numerous ways to do this programmatically. A simple way is to create a variable and put the result of your function and / or query into it.
    
2 Insert the message "ticketId X" where X is the id of your new ticket into Msg

3 Add a Brain Freeze product to this ticket. The choices for this product should include `vanilla` ice cream and `cherry` flavoring. You may programmatically or manually (visually) look up the product code and ingredient information (qty, unit, etc.) for a Brain Freeze. Keep track of the unique id of this product sold.
    
4 Insert the message "brain freeze X" where X is the id of your the productSold for the brain freeze into Msg.
    
5 Add a regular slush product with "grape" flavor.  You may programmatically or manually look up the product code  and ingredient information (qty, unit, etc.) for a regular slush.
    
6 Insert the message "regular slush X" where X is the id of your the productSold for the regular slush into Msg.
    
7 Commit the transaction.    

In [156]:
%%sql 

CREATE OR REPLACE FUNCTION insertData () 
  RETURNS VOID AS
$$
DECLARE
  newTicketID INTEGER;
  newProductSaleID INTEGER;
  newSaleRecordID1 INTEGER;
  newSaleRecordID2 INTEGER;
  ingredientAssemblyRecord RECORD;
  productRecord RECORD;
BEGIN
    -- creating a new ticket
    SELECT * 
    FROM createTicket(5) 
    INTO newTicketID;

    INSERT INTO msg (occurred, msg)
    VALUES (CURRENT_TIMESTAMP, 'ticketId ' || newTicketID);

    -- fetching the productRecord for brain freeze
    SELECT *
    FROM Product
    where productName = 'brain freeze'
    INTO productRecord;

    -- adding product to the ticket
    SELECT * 
    FROM addProductToTicket( newTicketID, productRecord.productCode )
    INTO newProductSaleID;
    
    -- getting ingredient1 info
    SELECT a.*
    FROM Assembly a
    where a.productCode = productRecord.productCode and a.ingName = 'ice cream'
    INTO ingredientAssemblyRecord;

    -- adding ingredient1 choice to the product
    SELECT * 
    FROM addIngChoiceToProductSold ( newProductSaleID, 'ice cream', ingredientAssemblyRecord.qty, ingredientAssemblyRecord.unit, 'vanilla')
    INTO newSaleRecordID1;

    -- getting ingredient2 info
    SELECT a.*
    FROM Assembly a
    where a.productCode = productRecord.productCode and a.ingName = 'flavoring'
    INTO ingredientAssemblyRecord;

    -- adding ingredient2 choice to the product
    SELECT * 
    FROM addIngChoiceToProductSold ( newProductSaleID, 'flavoring', ingredientAssemblyRecord.qty, ingredientAssemblyRecord.unit, 'cherry')
    INTO newSaleRecordID2;

    INSERT INTO msg (occurred, msg)
    VALUES (CURRENT_TIMESTAMP, 'brain freeze ' || newProductSaleID);

    -- fetching the productRecord for regular slush
    SELECT *
    FROM Product
    where productName = 'regular slush'
    INTO productRecord;

    -- adding product to the ticket
    SELECT * 
    FROM addProductToTicket( newTicketID, productRecord.productCode )
    INTO newProductSaleID;
    
    -- getting ingredient1 info
    SELECT a.*
    FROM Assembly a
    where a.productCode = productRecord.productCode and a.ingName = 'flavor'
    INTO ingredientAssemblyRecord;

    -- adding ingredient1 choice to the product
    SELECT * 
    FROM addIngChoiceToProductSold ( newProductSaleID, 'flavor', ingredientAssemblyRecord.qty, ingredientAssemblyRecord.unit, 'grape')
    INTO newSaleRecordID1;

    INSERT INTO msg (occurred, msg)
    VALUES (CURRENT_TIMESTAMP, 'regular slush ' || newProductSaleID);
END;
$$
LANGUAGE plpgsql;

START TRANSACTION;
DO $$
BEGIN
  PERFORM insertData();
END $$;
COMMIT;

 * postgresql+psycopg2://@/postgres
Done.
Done.
Done.
Done.


[]

12. (2 points for correct messages) Show us only the messages in your msg table, order by the time in which they were inserted.

In [158]:
%%sql py_var_msg <<
SELECT msg
FROM Msg
ORDER BY occurred;

 * postgresql+psycopg2://@/postgres
3 rows affected.
Returning data to local variable py_var_msg


In [159]:
# do not modify or delete this cell
df_q02 = py_var_msg.DataFrame() # Save the result of this task for grading purposes
df_q02

,msg
0,ticketId 1
1,brain freeze 1
2,regular slush 2


## Run grading script

Leave the next cell in your notebook. We will use it to run our grading script.

In [ ]:
%system
myCmd = 'psql ' + params['user'] + ' -d ' +params['dbname'] + ' -p ' + params['port'] + ' -o impSQLoutput.txt -f impSQLgrader.sql'
!{myCmd}

12. (9 points) Describe and provide SQL statements for 3 different test cases for `findLongestChain`, different from the examples provided above. Explain what each test case tests. 

Test case 1 description: The input event ID doesn not exists :=> output should be -1

In [160]:
%%sql
-- case 1: populating the data
CREATE OR REPLACE FUNCTION runTestCase1 () 
  RETURNS VOID AS
$$
DECLARE
  outputValue RECORD;
BEGIN
    SELECT * FROM findLongestChain (  0, 'mt') INTO outputValue;

    IF
      outputValue.findLongestChain = -1
    THEN
      RAISE NOTICE 'test case passed!';
    ELSE
      RAISE NOTICE 'test case failed!';
    END IF;
END;
$$
LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres
Done.


[]

In [161]:
#case 1: The input event ID doesn not exists :=> output should be -1
!psql postgres -c "SELECT runTestCase1();"

NOTICE:  test case passed!
 runtestcase1 
--------------
 
(1 row)



Test case 2 description: The input product code has no sales for that event :=> output should be 0

In [162]:
%%sql
CREATE OR REPLACE FUNCTION runTestCase2 () 
  RETURNS VOID AS
$$
DECLARE
  outputValue RECORD;
BEGIN
    SELECT * FROM findLongestChain (  5, 'mt') INTO outputValue;

    IF
      outputValue.findLongestChain = 0
    THEN
      RAISE NOTICE 'test case passed!';
    ELSE
      RAISE NOTICE 'test case failed!';
    END IF;
END;
$$
LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres
Done.


[]

In [163]:
# case 2: The input product code has no sales for that event :=> output should be 0
!psql postgres -c "SELECT runTestCase2();"

NOTICE:  test case passed!
 runtestcase2 
--------------
 
(1 row)



Test case 3 description: a normal case where the product has sales in the particular event. Each ticket has a single product assosciated to it.

In [164]:
%%sql
-- case 3: populating the data
CREATE OR REPLACE FUNCTION runTestCase3 () 
  RETURNS VOID AS
$$
DECLARE
  newTicketID INTEGER;
  productRecordBrainFreeze RECORD;
  productRecordRegularSlush RECORD;
  outputValue RECORD;
BEGIN

    -- creating a new event
    INSERT INTO TruckEvent (eventId, eventName, eventStart, eventPlannedEnd, eventStatus, menuName)
    VALUES (6, 'testing', '2021-12-01 20:00:00', '2021-12-02 12:00:00', 'Scheduled', 'Full menu');

    -- fetching the productRecord for brain freeze
    SELECT *
    FROM Product
    where productName = 'brain freeze'
    INTO productRecordBrainFreeze;

    -- fetching the productRecord for regular slush
    SELECT *
    FROM Product
    where productName = 'regular slush'
    INTO productRecordRegularSlush;

    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );

    SELECT * FROM findLongestChain (  6, 'bf') INTO outputValue;

    IF
      outputValue.findLongestChain = 4
    THEN
      RAISE NOTICE 'test case passed!';
    ELSE
      RAISE NOTICE 'test case failed!';
    END IF;

    DELETE FROM SaleRecord WHERE productSoldID IN (SELECT productSoldID FROM ProductSold WHERE ticketID IN (SELECT ticketID FROM Ticket WHERE eventID = 6));
    DELETE FROM ProductSold WHERE ticketID IN (SELECT ticketID FROM Ticket WHERE eventID = 6);
    DELETE FROM Ticket WHERE eventID = 6;
    DELETE FROM TruckEvent WHERE eventID = 6;
END;
$$
LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres
Done.


[]

In [165]:
# case 3: here, the result should be 4
!psql postgres -c "SELECT runTestCase3();"

NOTICE:  test case passed!
 runtestcase3 
--------------
 
(1 row)



Test Case 4: checking a normal case, A ticket can have multiple products!

In [166]:
%%sql
-- case 4: populating the data
CREATE OR REPLACE FUNCTION runTestCase4 () 
  RETURNS VOID AS
$$
DECLARE
  newTicketID INTEGER;
  productRecordBrainFreeze RECORD;
  productRecordRegularSlush RECORD;
  outputValue RECORD;
BEGIN

    -- creating a new event
    INSERT INTO TruckEvent (eventId, eventName, eventStart, eventPlannedEnd, eventStatus, menuName)
    VALUES (6, 'testing', '2021-12-01 20:00:00', '2021-12-02 12:00:00', 'Scheduled', 'Full menu');

    -- fetching the productRecord for brain freeze
    SELECT *
    FROM Product
    where productName = 'brain freeze'
    INTO productRecordBrainFreeze;

    -- fetching the productRecord for regular slush
    SELECT *
    FROM Product
    where productName = 'regular slush'
    INTO productRecordRegularSlush;

    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    SELECT createTicket(6) INTO newTicketID;
    perform addProductToTicket( newTicketID , productRecordRegularSlush.productCode );
    perform addProductToTicket( newTicketID , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordBrainFreeze.productCode );
    perform addProductToTicket( createTicket(6) , productRecordRegularSlush.productCode );

    SELECT * FROM findLongestChain (  6, 'bf') INTO outputValue;

    IF
      outputValue.findLongestChain = 4
    THEN
      RAISE NOTICE 'test case passed!';
    ELSE
      RAISE NOTICE 'test case failed!';
    END IF;

    DELETE FROM SaleRecord WHERE productSoldID IN (SELECT productSoldID FROM ProductSold WHERE ticketID IN (SELECT ticketID FROM Ticket WHERE eventID = 6));
    DELETE FROM ProductSold WHERE ticketID IN (SELECT ticketID FROM Ticket WHERE eventID = 6);
    DELETE FROM Ticket WHERE eventID = 6;
    DELETE FROM TruckEvent WHERE eventID = 6;
END;
$$
LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres
Done.


[]

In [174]:
%%sql
SELECT * FROM TruckEvent;

 * postgresql+psycopg2://@/postgres
1 rows affected.


eventid,eventname,eventstart,eventplannedend,eventactualend,eventstatus,menuname,totaldollarsales,totalnumproducts,totalnumtickets
5,DB study group,2021-12-01 20:00:00,2021-12-02 12:00:00,None,Scheduled,Full menu,10.00,2,1


In [167]:
# case 3: here, the result should be 4
!psql postgres -c "SELECT runTestCase4();"

NOTICE:  test case passed!
 runtestcase4 
--------------
 
(1 row)



13. (5 points) Add code to drop functions, custom views, new tables, and triggers that you created

Be sure to put the drop statements in the correct order (or use CASCADE)!


In [141]:
%%sql 
DROP FUNCTION IF EXISTS runTestCase4;
DROP FUNCTION IF EXISTS runTestCase3;
DROP FUNCTION IF EXISTS runTestCase2;
DROP FUNCTION IF EXISTS runTestCase1;
DROP FUNCTION IF EXISTS insertData;
DROP TRIGGER IF EXISTS updateTicket ON ProductSold;
DROP FUNCTION IF EXISTS updateTicketInfo;
DROP FUNCTION IF EXISTS findLongestChain;
DROP FUNCTION IF EXISTS updateEventSales;
DROP FUNCTION IF EXISTS addIngChoiceToProductSold;
DROP FUNCTION IF EXISTS addProductToTicket;
DROP FUNCTION IF EXISTS createTicket;
DROP VIEW IF EXISTS eventIngredient;

 * postgresql+psycopg2://@/postgres
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]